In [1]:
!pip install -Uqq ipywidgets

In [2]:
!pip install -Uqq sentence-transformers chromadb scikit-build numpy opencv-python unstructured-inference langchain langchain-community langchain-core unstructured

In [3]:
!pip install -Uqq "gradio==3.50"

In [4]:
# the warnings in the libraries used can get old, do not show them
import warnings
warnings.filterwarnings('ignore')

In [5]:
!wget -O index.html "https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14"

--2024-02-05 11:51:45--  https://access.redhat.com/documentation/en-us/openshift_container_platform/4.14
Resolving access.redhat.com (access.redhat.com)... 23.216.132.69, 23.216.132.70, 2600:141b:e800:1d::17d8:848d, ...
Connecting to access.redhat.com (access.redhat.com)|23.216.132.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html’

index.html              [ <=>                ] 131.28K  --.-KB/s    in 0.1s    

2024-02-05 11:51:46 (922 KB/s) - ‘index.html’ saved [134431]



In [6]:
import re

with open("index.html", "r") as f:
    html = f.read()

urls = []

for x in html.splitlines():
    if "Single-page HTML" in x:
        m = re.match(".*href=\"([^\"]*)\".*", x)
        urls.append(f"https://access.redhat.com{m.groups()[0]}")

In [7]:
len(urls)

41

In [8]:
from transformers import pipeline
model= pipeline(model="google/flan-t5-large") #'text2text-generation'
model.save_pretrained("flan-t5-large")

In [9]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(model_id="flan-t5-large", task="text2text-generation", model_kwargs={"temperature":1e-10})

In [10]:
from langchain_community.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(urls=urls)

pages = loader.load_and_split()

In [11]:
len(pages)

6313

In [12]:
from langchain_community.embeddings import HuggingFaceEmbeddings
import intel_extension_for_pytorch as ipex

model_name = "sentence-transformers/all-mpnet-base-v2"
encode_kwargs = {'normalize_embeddings': False}

working_devices=[]

for device in "cpu", "cuda", "ipu", "xpu", "mkldnn", "opengl", "opencl", "ideep", "hip", "ve", "fpga", "ort", "xla", "lazy", "vulkan", "mps", "meta", "hpu", "mtia", "privateuseone":
    model_kwargs = {'device': device}
    print(f"Trying device={device}")
    try:
        hf = HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
    except Exception as e:
        print(e)
        continue
    working_devices.append(device)
print(f"Working devices={working_devices}")

2024-02-05 11:54:21,825 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Trying device=cpu


2024-02-05 11:54:22,982 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Trying device=cuda


2024-02-05 11:54:24,520 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


...

Trying device=hpu


2024-02-05 11:54:42,584 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


PyTorch is not linked with support for hpu devices
Trying device=mtia


2024-02-05 11:54:43,696 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


PyTorch is not linked with support for mtia devices
Trying device=privateuseone
PyTorch is not linked with support for privateuseone devices
Working devices=['cpu', 'meta']


In [14]:
# https://api.python.langchain.com/en/latest/embeddings/langchain_community.embeddings.huggingface.HuggingFaceEmbeddings.html

from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

model_name = "sentence-transformers/all-mpnet-base-v2"

device = ("cuda" if torch.cuda.is_available() else "cpu")

model_kwargs = {'device': device}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

2024-02-05 11:56:04,017 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


In [16]:
from langchain_community.vectorstores import FAISS

%time vectorestore = FAISS.from_documents(pages, hf)

Batches:   0%|          | 0/198 [00:00<?, ?it/s]

2024-02-05 14:40:37,695 - faiss.loader - INFO - Loading faiss with AVX2 support.
2024-02-05 14:40:37,747 - faiss.loader - INFO - Successfully loaded faiss with AVX2 support.


CPU times: user 2h 59min 57s, sys: 35min 51s, total: 3h 35min 48s
Wall time: 55min 12s


In [17]:
%time vectorestore.save_local("ocp-llm-rag-vectorstore")

CPU times: user 31.3 ms, sys: 29.1 ms, total: 60.4 ms
Wall time: 60.6 ms
